# Primer problema sencillo:

**Variables de decisión:**  
$x_i \in \{0,1\}$ Indica si el AP $i$ está encendido  
$a_{ij} \in \{0,1\}$ Indica si el cliente $j$ se conecta al AP $i$

**Parámetros:**  
$rssi_{ij}$ RSSI entre AP $i$ y cliente $j$  
$\text{umbral}$ Nivel mínimo de RSSI para conexión  
$\text{capacidad}_i$ Máximo de clientes del AP $i$  
$\text{edificio}_i$ Edificio del AP $i$

**Función objetivo:**  
$$
R = 
\min \sum_i \left( \frac{1}{\text{capacidad}_i} \sum_j a_{ij} \right)^2
$$  
*(Minimizar la congestión total ponderada)*

**Restricciones:**  
<!-- $$
\sum_i a_{ij} = 1 \quad \forall j
$$  
*(Cada cliente se conecta a un único AP)*   -->

$$
a_{ij} = 0 \quad \text{si } rssi_{ij} < \text{umbral}
$$  
*(Solo conexión con buena señal)*  

$$
a_{ij} \leq x_i \quad \forall i,j
$$  
*(Solo a APs encendidos)*  

$$
\sum_j a_{ij} \leq \text{capacidad}_i \quad \forall i
$$  
*(Respetar capacidad de cada AP)*

In [6]:
import gzip
import pandas as pd

# Ajusta la ruta a tu archivo real
with gzip.open('../datos_ceibal/datos_resto_del_anio/datos_APs_Abril.csv.gz', 'rt') as f:
    # Si deseas muestrear solo primeras filas para pruebas:
    df = pd.read_csv(f) #, nrows=1000
    # O comenta nrows para cargar todo:

In [7]:
# Ajusta la ruta al archivo de mapeo real
df_mapping = pd.read_csv('../graphs/from_mac_hexa_to_building_id.csv')
# Debe tener columnas: 'MAC_AP_hexa' y 'building_id'


In [8]:
# Ejemplo de limpieza básica:
df['MAC_AP_hexa'] = df['MAC_AP_hexa'].str.strip().str.upper()
df_mapping['MAC_AP_hexa'] = df_mapping['MAC_AP_hexa'].str.strip().str.upper()

In [ ]:
# Merge izquierdo: conservas todos los registros de df y añades building_id si existe en mapping
df_merged = df.merge(df_mapping, on='MAC_AP_hexa', how='left')

In [11]:
target_building = 994  # Reemplaza por el ID real que te interese
df_building = df_merged[df_merged['building_id'] == target_building].copy()

df_building['timestamp'] = pd.to_datetime(df_building['timestamp'])

In [12]:
# Conteo por banda
print("Conteo por banda:")
print(df_building['Banda'].value_counts())

# Conteo por canal en cada banda
for banda in df_building['Banda'].unique():
    df_band = df_building[df_building['Banda'] == banda]
    print(f"\nBanda {banda}: canales y sus conteos")
    print(df_band['Canal'].value_counts().sort_index())

# Distribución de Tx_power
print("\nResumen de Tx_power (dBm):")
print(df_building['Tx_power'].describe())

# Si Power_level requiere conversión, define función de conversión:
# def nivel_a_dBm(modelo, nivel): ...
# df_building['Tx_dBm_calc'] = df_building.apply(lambda row: nivel_a_dBm(row['Modelo'], row['Power_level']), axis=1)


Conteo por banda:
Banda
0    5384
1    5384
Name: count, dtype: int64

Banda 0: canales y sus conteos
Canal
1.0     1515
6.0     1595
11.0    2255
Name: count, dtype: int64

Banda 1: canales y sus conteos
Canal
36.0     1358
40.0     1233
44.0     1135
52.0       37
56.0      166
60.0      128
64.0      212
149.0     779
157.0     127
161.0     190
Name: count, dtype: int64

Resumen de Tx_power (dBm):
count    10688.000000
mean        13.240644
std          3.449770
min          7.000000
25%         10.000000
50%         13.000000
75%         14.000000
max         23.000000
Name: Tx_power, dtype: float64


In [14]:
import networkx as nx

# Suponiendo df_building tiene columnas 'MAC_AP_hexa', 'Banda', 'Canal'
G = nx.Graph()
# Añadir nodos con atributos
for _, row in df_building.iterrows():
    ap = row['MAC_AP_hexa']
    G.add_node(ap, banda=row['Banda'], canal=row['Canal'])

# Función para determinar si dos canales interfieren:
def interfieren(banda, canal1, canal2):
    if banda == 0:  # 2.4 GHz: canales 1-14, solapamiento
        # Ejemplo simple: considerar interferencia si |canal1 - canal2| < 5
        return abs(canal1 - canal2) < 5
    else:  # 5 GHz: más canales, menos solapamiento; ajustar según anchura de canal (20/40/80 MHz)
        # Como simplificación, considerar interferencia solo si canal idéntico
        return canal1 == canal2

# Añadir aristas
aps = list(df_building['MAC_AP_hexa'].unique())
for i in range(len(aps)):
    for j in range(i+1, len(aps)):
        ap1, ap2 = aps[i], aps[j]
        row1 = df_building[df_building['MAC_AP_hexa'] == ap1].iloc[0]
        row2 = df_building[df_building['MAC_AP_hexa'] == ap2].iloc[0]
        if row1['Banda'] == row2['Banda']:
            if interfieren(row1['Banda'], row1['Canal'], row2['Canal']):
                G.add_edge(ap1, ap2)
# Ahora G muestra relaciones de interferencia aproximada.

nx.write_gexf(G, "grafo_interferencia_buildingX.gexf")



## II‑bis. Selección de Canal en 2.4 GHz

**Nuevas variables de decisión** (por cada slot $t$):  
- Para cada transmisor $i$ y canal $k \in \mathcal C = \{1,\dots,11\}$,  
  $$z_{i,k}(t) = 
    \begin{cases}
      1, &\text{si la AP }i\text{ usa el canal }k\text{ en el slot }t,\\
      0, &\text{en otro caso.}
    \end{cases}$$  
- Se impone unicidad:  
  $$\sum_{k\in\mathcal C} z_{i,k}(t) = 1,\quad \forall i,t.$$

**Variables originales** (potencia) y definición de tasa $r_i$:  
- Matriz de canales $H(t)$ y estados de nodo $x(t)$ como en (1) :contentReference[oaicite:0]{index=0}.  
- Política de potencia $p_i(t) = p_i\bigl(H(t),x(t)\bigr)\in\{0,p_0\}$.

**Función objetivo extendida**:  
$$
\max_{p(\cdot),\,z(\cdot)}\;
\sum_{t=1}^T\sum_{i=1}^m 
  \Bigl[
    r_i\bigl(H(t),p(t)\bigr)
    - \lambda \sum_{k\in\mathcal C} U_k \,z_{i,k}(t)
  \Bigr],
$$  
donde $U_k$ cuantifica la ocupación promedio del canal $k$ (p.ej. % de utilización medido por sniffer).

### Restricciones

1. **Capacidad y potencia** (igual que en (1))  
   $$r_i = \mathbb{E}\Bigl[\log\bigl(1 + \tfrac{|h_{ii}|^2\,p_i}{\sigma^2 + \sum_{j\neq i}|h_{ji}|^2\,p_j}\bigr)\Bigr],$$  
   $$\mathbb{E}\Bigl[\sum_i p_i\Bigr] \le P_{\max},\quad p_i\in\{0,p_0\}.$$  
   *(ver :contentReference[oaicite:1]{index=1})*

2. **Unicidad de canal**  
   $$\sum_{k\in\mathcal C} z_{i,k}(t) = 1,\quad z_{i,k}(t)\in\{0,1\},\quad \forall i,k,t.$$

3. **Evitar colisión con el vecino más cercano**  
   Definimos para cada $i$ su vecino crítico  
   $$j_i^* = \arg\max_{j\neq i} A_{ij},$$  
   donde $A_{ij}$ es la atenuación de $i$ a $j$. Luego imponemos  
   $$z_{i,k}(t) + z_{j_i^*,\,k}(t) \le 1,\quad \forall i,k,t.$$

4. **(Opcional) Forzar sólo canales no solapados**  
   Si se desea restringir a $\{1,6,11\}$, basta con fijar  
   $$z_{i,k}(t) = 0,\quad \forall k\notin\{1,6,11\}.$$

---

> **Notas de implementación**  
> - La constante $\lambda\ge0$ ajusta la penalización por canales muy ocupados.  
> - Los valores de ocupación $U_k$ y la matriz de atenuaciones $A_{ij}$ se calculan en preprocesamiento :contentReference[oaicite:2]{index=2}.  
> - Esta extensión preserva la estructura de política aleatoria con REINFORCE, añadiendo salidas $z_{i,k}$ a la red y un término extra en el objetivo.  


1. **Variables nuevas**

   * $z_{i,k}(t)$: decide **qué canal** usa la AP $i$ en el slot $t$.
   * Sigue existiendo la variable de potencia $p_i(t)$.

2. **Objetivo modificado**

   $$
     \max\;\sum_{t,i}\;\Bigl[\;r_i\bigl(H(t),p(t)\bigr)\;-\;\lambda\,\sum_k U_k\,z_{i,k}(t)\Bigr].
   $$

   * El primer término busca **maximizar la tasa** de datos $r_i$.
   * El segundo término **penaliza** (resta) un valor proporcional a $U_k$ si eliges un canal muy usado.
   * El parámetro $\lambda$ ajusta cuánto peso le das a esa penalización.

3. **Restricción de vecinos**

   * Para evitar interferir con tu AP más crítica, imponemos

     $$
       z_{i,k}(t) + z_{j_i^*,k}(t) \;\le\;1
       \quad\forall i,k,t,
     $$

     donde $j_i^*$ es el “vecino” de $i$ con mayor atenuación recibida (más propenso a interferir).
   * Así, $i$ y su vecino fuerte **nunca comparten canal**.

4. **Ciclo de ejecución**

   1. **Preprocesas**: calculas $U_k$ y la matriz de atenuaciones $A_{ij}$.
   2. **En cada slot** $t$: tu agente (GNN+REINFORCE) elige un par $\{p_i(t),\,k_i(t)\}$ para cada AP, respetando las restricciones.
   3. **Aprendes**: el término $-\lambda\sum U_k z_{i,k}$ le enseña a la red a favorecer canales menos usados.
